In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder

In [18]:
data = pd.read_csv('fraud.csv')
data.head()


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [20]:
# drop unique and string coloumns
data_new=data.drop('isFraud',axis=1)
data_new.drop('nameOrig',axis=1,inplace=True)
data_new.drop('nameDest',axis=1,inplace=True)
data_target=data['isFraud']
data_target

0          0
1          0
2          1
3          1
4          0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 6362620, dtype: int64

In [21]:
#convert type from string to numbers
ord_enc = OrdinalEncoder()
data_new["type"] = ord_enc.fit_transform(data_new[["type"]])
data_new


,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud
0,1,3.0,9839.64,170136.00,160296.36,0.00,0.00,0
1,1,3.0,1864.28,21249.00,19384.72,0.00,0.00,0
2,1,4.0,181.00,181.00,0.00,0.00,0.00,0
3,1,1.0,181.00,181.00,0.00,21182.00,0.00,0
4,1,3.0,11668.14,41554.00,29885.86,0.00,0.00,0
...,...,...,...,...,...,...,...,...
6362615,743,1.0,339682.13,339682.13,0.00,0.00,339682.13,0
6362616,743,4.0,6311409.28,6311409.28,0.00,0.00,0.00,0
6362617,743,1.0,6311409.28,6311409.28,0.00,68488.84,6379898.11,0
6362618,743,4.0,850002.52,850002.52,0.00,0.00,0.00,0


In [24]:
polynomial_regression_meansquarerrors=[]
#Generate polynomial and features
for i in range(1,3):
    poly = PolynomialFeatures(degree= i, include_bias=False)
    polynomial_features = poly.fit_transform(data_new)
    
    # Divide dataset after importing into Train/Cross Validation/Test
    X_train, X_test, y_train, y_test = train_test_split(polynomial_features, data_target, test_size=0.4, random_state=1)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

    #logistic regression
    polynomialregression_model = LogisticRegression()
    polynomialregression_model.fit(X_train, y_train)
    ypredicted_polynomialregression = polynomialregression_model.predict(X_val)
    polynomial_regression_meansquarerror = np.sqrt(mean_squared_error(y_val, ypredicted_polynomialregression))
    polynomial_regression_meansquarerrors.append(polynomial_regression_meansquarerror)

#print minimum mean square error to get the best degree
x=np.array(polynomial_regression_meansquarerrors)
print('the mean square error array is ' ,x) 
print('the mimimum value is' , np.min(polynomial_regression_meansquarerrors))
print('the minimum index of minimum value is ', np.argmin(polynomial_regression_meansquarerrors))
print('the best degree is ',np.argmin(polynomial_regression_meansquarerrors)+1)

the mean square error array is  [0.0424399  0.03625893]
the mimimum value is 0.036258929630713754
the minimum index of minimum value is  1
the best degree is  2


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
# after getting the best DEGREE

#Generate polynomial and features
poly = PolynomialFeatures(degree= np.argmin(polynomial_regression_meansquarerrors)+1, include_bias=False)
polynomial_features = poly.fit_transform(data_new)

# Divide dataset after importing into Train/Cross Validation/Test
X_train, X_test, y_train, y_test = train_test_split(polynomial_features, data_target, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

#logistic regression
polynomialregression_model = LogisticRegression()
polynomialregression_model.fit(X_train, y_train)
ypredicted_polynomialregression = polynomialregression_model.predict(X_test)
polynomial_regression_meansquarerror = np.sqrt(mean_squared_error(y_test, ypredicted_polynomialregression))
print(polynomial_regression_meansquarerror)

0.03553652418932332


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
# Stratified K-Fold 

from sklearn.model_selection import StratifiedKFold
from statistics import mean, stdev

stratified_kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)
stratifiedArray = []
  
for train_index, test_index in stratified_kfold.split(polynomial_features, data_target):
    x_train_fold, x_test_fold = polynomial_features[train_index], poly_features[test_index]
    y_train_fold, y_test_fold = data_target[train_index], data_target[test_index]
    polynomialregression_model = LogisticRegression()
    polynomialregression_model.fit(x_train_fold, y_train_fold)
    stratifiedArray.append(polynomialregression_model.score(x_test_fold, y_test_fold))
  
print('the possible accuracies are :', stratifiedArray)
print('\nMaximum Accuracy from this model is:',max(stratifiedArray)*100, '%')
print('\nMinimum Accuracy is:',min(stratifiedArray)*100, '%')
print('\n0verall Accuracy is:',mean(stratifiedArray)*100, '%')
print('\nStandard Deviation is:', stdev(stratifiedArray))

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


the possible accuracies are : [0.9987093367197789, 0.9987093367197789, 0.9987093367197789, 0.9987087080479425]

Maximum Accuracy from this model is: 99.8709336719779 %

Minimum Accuracy is: 99.87087080479425 %

0verall Accuracy is: 99.87091795518198 %

Standard Deviation is: 3.143359181834171e-07


In [28]:
#Apply Regularization
from sklearn.linear_model import LogisticRegression
result=[]
poly = PolynomialFeatures(degree= 2, include_bias=False)
polynomial_features = poly.fit_transform(data_new)
X_train, X_test, y_train, y_test = train_test_split(poly_features, data_target, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)
  
polynomialregression_model = LogisticRegression(penalty='l1', C=1 ,solver='saga', max_iter=2)
polynomialregression_model.fit(X_train, y_train)
ypredicted_polynomialregression = polynomialregression_model.predict(X_test)
polynomial_regression_meansquarerror = np.sqrt(mean_squared_error(y_test, ypredicted_polynomialregression))
result.append(polynomial_regression_meansquarerror)
print(result)

[0.03553652418932332]


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
